# Setup Work Environment

Run the following cell to setup all the dependencies and the code. There should be no changes required from this cell.

Don't forget to ensure the GPU has already attached to your working environment. You can check it in `Runtime -> Manage sessions -> Check if word GPU is available next to the notebook's name`, you can also double check in `Runtime -> Change runtime type -> Check if GPU has already selected from the dropdown menu`

In [1]:
!rm -rf xib
!pip install pytrie enlighten colorlog inflection ipapy
!git clone https://github.com/akurniawan/xib.git
!cd xib && git clone https://github.com/j-luo93/dev_misc.git && cd dev_misc && git checkout b44fde842a6311e03f731cd4e110dcd9fc394db7 && pip install -e .
!cd xib && pip install -e .

     |████████████████████████████████| 102kB 2.5MB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
     |████████████████████████████████| 81kB 5.5MB/s 
  Created wheel for pytrie: filename=PyTrie-0.4.0-cp37-none-any.whl size=6089 sha256=8cae306a9e3c736823bce725ca29d7e3d8c626dc09b6cac27e52a855de533004
  Stored in directory: /root/.cache/pip/wheels/ef/2a/41/5870cad27097f3b3d7b3d96aa5897d502db08cafba9051bd62
  Created wheel for ipapy: filename=ipapy-0.0.9.0-cp37-none-any.whl size=38724 sha256=737ba5d70b9ad117f16b7dcab773ef1eede7168c7eaaab0e618c783dadb0d90b
  Stored in directory: /root/.cache/pip/wheels/d1/f3/00/9f83f4dada00246acda572d4248dc8215c5a8ca37bb4f173a8
Successfully built pytrie ipapy
Cloning into 'xib'...
remote: Enumerating objects: 3093, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 3093 (delta 32), reused 47 (delta 24), pack-reused 3032
Receiving objects: 100% (3093/3093), 636.08 KiB | 1.16 MiB/s, 

# Setup Dataset

There are 2 ways to setup your dataset:
1. Mount Gdrive to Colab environment. If you decided to go with this and would like to access the data directly in our `LCT Project` shared folder, you can follow instruction in https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab to load `Preprocessed file` folder in Shared google drive. Essentially you just have to go to the location of the folder, right click and choose `Add a shortcut to Drive`. After that you just have to run the cell below
2. Upload your dataset to `sample_data` folder in google colab environment. Be aware that you **will** lose your data in this folder when you restart colab's environment

Please do remember that the word **must** be in an alphabetical or IPA form, that means no number, no non-alphabetical characters, etc. Otherwise, it will throw an error. If you're unsure whether your data is correct or not, run the last cell before **Run training** section to check whethere all your vocabs are valid


In [14]:
from ipapy.ipastring import IPAString
from ipapy import is_valid_ipa


dia2char = {
    'low': {'à': 'a', 'è': 'e', 'ò': 'o', 'ì': 'i', 'ù': 'u', 'ѐ': 'e', 'ǹ': 'n', 'ỳ': 'y'},
    'high': {'á': 'a', 'é': 'e', 'ó': 'o', 'ú': 'u', 'ý': 'y', 'í': 'i', 'ḿ': 'm', 'ĺ': 'l',
             'ǿ': 'ø', 'ɔ́': 'ɔ', 'ɛ́': 'ɛ', 'ǽ': 'æ', 'ə́': 'ə', 'ŕ': 'r', 'ń': 'n'},
    'rising_falling': {'ã': 'a'},
    'falling': {'â': 'a', 'î': 'i', 'ê': 'e', 'û': 'u', 'ô': 'o', 'ŷ': 'y', 'ĵ': 'j'},
    'rising': {'ǎ': 'a', 'ǐ': 'i', 'ǔ': 'u', 'ǒ': 'o', 'ě': 'e'},
    'extra_short': {'ă': 'a', 'ĕ': 'e', 'ĭ': 'i', 'ŏ': 'o', 'ŭ': 'u'},
    'nasalized': {'ĩ': 'i', 'ũ': 'u', 'ã': 'a', 'õ': 'o', 'ẽ': 'e', 'ṽ': 'v', 'ỹ': 'y'},
    'breathy_voiced': {'ṳ': 'u'},
    'creaky_voiced': {'a̰': 'a', 'ḭ': 'i', 'ḛ': 'e', 'ṵ': 'u'},
    'centralized': {'ë': 'e', 'ä': 'a', 'ï': 'i', 'ö': 'o', 'ü': 'u', 'ÿ': 'y'},
    'mid': {'ǣ': 'æ', 'ū': 'u', 'ī': 'i', 'ē': 'e', 'ā': 'a', 'ō': 'o'},
    'voiceless': {'ḁ': 'a'},
    'extra_high': {'ő': 'o'},
    'extra_low': {'ȁ': 'a'},
    'syllabic': {'ạ': 'a', 'ụ': 'u'}
}


dia2code = {
    'low': 768,
    'high': 769,
    'rising_falling': 771,
    'falling': 770,
    'rising': 780,
    'extra_short': 774,
    'nasalized': 771,
    'breathy_voiced': 804,
    'creaky_voiced': 816,
    'centralized': 776,
    'mid': 772,
    'voiceless': 805,
    'extra_high': 779,
    'extra_low': 783,
    'syllabic': 809,
    'high_rising': 7620,
    'low_rising': 7621,
}


char2ipa_char = dict()
for dia, char_map in dia2char.items():
    code = dia2code[dia]
    s = chr(code)
    for one_char, vowel in char_map.items():
        char2ipa_char[one_char] = vowel + s


to_remove = {'ᶢ', '̍', '-', 'ⁿ', 'ᵑ', 'ᵐ', 'ᶬ', ',', 'ᵊ', 'ˢ', '~', '͍', 'ˣ', 'ᵝ', '⁓', '˭', 'ᵈ', '⁽', '⁾', '˔', 'ᵇ',
             '+', '⁻'}


def clean(s):
    if s == '◌̃':
        return ''
    return ''.join(c for c in s if c not in to_remove)


def sub(s):
    return ''.join(char2ipa_char.get(c, c) for c in s)


to_standardize = {
    'ˁ': 'ˤ',
    "'": 'ˈ',
    '?': 'ʔ',
    'ṭ': 'ʈ',
    'ḍ': 'ɖ',
    'ṇ': 'ɳ',
    'ṣ': 'ʂ',
    'ḷ': 'ɭ',
    ':': 'ː',
    'ˇ': '̌',
    'ỵ': 'y˞',
    'ọ': 'o˞',
    'ř': 'r̝',  # Czech
    '͈': 'ː',  # Irish
    'ŕ̩': sub('ŕ') + '̩',  # sanskrit
    'δ': 'd',  # Greek
    'ń̩': sub('ń') + '̩',  # unsure
    'ε': 'e',
    'X': 'x',
    'ṍ': sub('õ') + chr(769),
    'ÿ̀': sub('ÿ') + chr(768),
    '∅': 'ʏ'  # Norvegian,
}


def get_string(s: str) -> IPAString:
    return IPAString(unicode_string=clean(sub(standardize(s))))


def standardize(s):
    return ''.join(to_standardize.get(c, c) for c in s)

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# Set your dataset path here
KNOWN_LANG_PATH = "/content/sample_data/it_testing.txt"
UNKNOWN_LANG_PATH = "/content/sample_data/srb_lat.txt"

In [22]:
print("Check KNOWN Language file format")
with open(KNOWN_LANG_PATH, "r", encoding="utf8") as f:
    for line in f.readlines():
        s = clean(sub(standardize(line.strip())))
        # print(s, is_valid_ipa(s))
        if not is_valid_ipa(s):
            print(s, "is invalid")


print("\nCheck UNKNOWN_LANG_PATH Language file format")
with open(UNKNOWN_LANG_PATH, "r", encoding="utf8") as f:
    for line in f.readlines():
        s = clean(sub(standardize(line.strip())))
        # print(s, is_valid_ipa(s))
        if not is_valid_ipa(s):
            print(s, "is invalid")

Check KNOWN Language file format
dona39signor is invalid
acquietata8928ritorn is invalid
accetta8929entr is invalid
servi8930entr is invalid

Check UNKNOWN_LANG_PATH Language file format


# Run Training

As of now, the training will run indefinitely and not sure if changing this will affect the rest of the code. For that reason, we need to stop the training manually once we feel that the losses are no longer improving. We can monitor the `ll` variable from script output inside a table with the following format to know when to stop the training (i.e. the `ll` is close to zero)

```
+----------------------------------------+  
|                  3_8                   |  
+-----------+----------+--------+--------+  
| name      | value    | weight | mean   |  
+-----------+----------+--------+--------+  
| grad_norm | 58.492   | 60     | 0.975  |  
| ll        | -336.246 | 60     | -5.604 |  
| reg       | 0.555    | 60     | 0.009  |  
+-----------+----------+--------+--------+
```

The other way to know when to stop the training is to also monitor the output of the model validation. Go to the next section to see how to analyze the result.

In [4]:
# Total number of phonetic feature groups
NUM_FEATURE_GROUPS = 10

# Total number of phonetic features
NUM_FEATURES = 10

# Initial value of threshold to determine whether two words are matched. This will determine
# whether two words are in match. The bigger the value, the more false positive we will have.
# However, if the value is too low, the model will not output anything
THRESHOLD = 1.5

# Cost in doing insertion and deletion operation in edit distance algorithm, refer to the paper for more details
INS_DEL_COST = 100.0

# Learning rate for adam optimizer
LR = 0.002

# How many training steps to do before running the evaluation steps
EVAL_INTERVAL = 500

In [24]:
!PYTHONPATH=/content/xib && /usr/local/bin/python -m xib.main --task extract \
  --vocab_path {KNOWN_LANG_PATH} --data_path {UNKNOWN_LANG_PATH} \
  --dim 112 --min_word_length 1 --max_word_length 10 --input_format text \
  --dense_input --eval_interval 20 \
  --char_per_batch 128 --gpus 0 \
  --num_feature_groups {NUM_FEATURE_GROUPS} --num_features {NUM_FEATURES} \
  --init_threshold {THRESHOLD} --init_ins_del_cost {INS_DEL_COST} --learning_rate {LR}

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/content/xib/dev_misc/dev_misc/utils.py:36: DeprecationWarning: Class/function Trainer deprecated.
  warnings.warn(message, warning_cls)
/content/xib/dev_misc/dev_misc/utils.py:36: RuntimeWarning: Class/function DecipherEsNoisyItalianP5Test is buggy.
  warnings.warn(message, warning_cls)
/usr/local/lib/python3.7/dist-packages/ipapy/ipastring.py:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableSequence
/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
INFO - 05/31/21 14:

# Analyzing The Result

The result will be in the following file `log/<DATE>/default/<TIME>/predictions/extract.<EPOCH>_<STEPS>.tsv`

For some reason, google colab won't show anything under `log` folder, so I would suggest to analyze it via `cat` command or download the result to your local computer and analyze it from there.

The content inside of the `tsv` file will consist of 4 different columns: `segment`, `ground_truth`, `prediction`, `matched_segment`. From my understanding, `segment` is the original segment of the unknown language; `ground_truth` similar to `segment` but with their exact index locations; `prediction` is the vocabulary prediction in the known language; and `matched_segment` is the information on which segment the unknown language match the vocabulary in known language. If you want, you can get more details by looking at the code in `evaluator.py` line 257


In [ ]:
# You can run the following command **sequentially**
# !ls log/ # to get the list of experiment dates
# !ls log/<date>/default/ # to get the list of experiment times, first replace date from the command above
# !ls log/<date>/default/<time>/predictions/<filename> # to get the list of filename, first replace date and time from the commands above

# Replace <date>, <time>, and <filename> from the commands above
# !cat log/<date>/default/<time>/predictions/<filename>

In [ ]:
# Or you can run the following command to download the result to your local environment
from google.colab import files

files.download("log/2021-05-29/default/09-15-25/predictions/extract.1_10.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The other way around is to direct the output to your google drive folder by setting up `--log_dir` parameter on the script. However, by setting up this parameter we won't have the same directory structure format as if we don't set the parameter.